In [ ]:
from argparse import Namespace
from nltk.tokenize import word_tokenize
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/CSC-project/PyTorchNLPBook/
!pip install -r requirements.txt

In [ ]:
args = Namespace(
    source_data_path="data/nmt/eng-fra.txt",
    output_data_path="data/nmt/simplest_eng_fra.csv",
    perc_train=0.7,
    perc_val=0.15,
    perc_test=0.15,
    seed=1337
)

assert args.perc_test > 0 and (args.perc_test + args.perc_val + args.perc_train == 1.0)

In [ ]:
with open(args.source_data_path) as fp:
    lines = fp.readlines()
    
lines = [line.replace("\n", "").lower().split("\t") for line in lines]

In [ ]:
data = []
for english_sentence, french_sentence in lines:
    data.append({"english_tokens": word_tokenize(english_sentence, language="english"),
                 "french_tokens": word_tokenize(french_sentence, language="french")})

In [ ]:
filter_phrases = (
    ("i", "am"), ("i", "'m"), 
    ("he", "is"), ("he", "'s"),
    ("she", "is"), ("she", "'s"),
    ("you", "are"), ("you", "'re"),
    ("we", "are"), ("we", "'re"),
    ("they", "are"), ("they", "'re")
)


In [ ]:
data_subset = {phrase: [] for phrase in filter_phrases}
for datum in data:
    key = tuple(datum['english_tokens'][:2])
    if key in data_subset:
        data_subset[key].append(datum)

In [ ]:
counts = {k: len(v) for k,v in data_subset.items()}
counts, sum(counts.values())

({('i', 'am'): 805,
  ('i', "'m"): 4760,
  ('he', 'is'): 1069,
  ('he', "'s"): 787,
  ('she', 'is'): 504,
  ('she', "'s"): 316,
  ('you', 'are'): 449,
  ('you', "'re"): 2474,
  ('we', 'are'): 181,
  ('we', "'re"): 1053,
  ('they', 'are'): 194,
  ('they', "'re"): 470},
 13062)

In [ ]:
np.random.seed(args.seed)

dataset_stage3 = []
for phrase, datum_list in sorted(data_subset.items()):
    np.random.shuffle(datum_list)
    n_train = int(len(datum_list) * args.perc_train)
    n_val = int(len(datum_list) * args.perc_val)

    for datum in datum_list[:n_train]:
        datum['split'] = 'train'
        
    for datum in datum_list[n_train:n_train+n_val]:
        datum['split'] = 'val'
        
    for datum in datum_list[n_train+n_val:]:
        datum['split'] = 'test'
    
    dataset_stage3.extend(datum_list)    

In [ ]:
# here we pop and assign into the dictionary, thus modifying in place
for datum in dataset_stage3:
    datum['source_language'] = " ".join(datum.pop('english_tokens'))
    datum['target_language'] = " ".join(datum.pop('french_tokens'))

In [ ]:
nmt_df = pd.DataFrame(dataset_stage3)

In [ ]:
nmt_df.head()

,source_language,split,target_language
0,he 's the cutest boy in town .,train,c'est le garçon le plus mignon en ville .
1,he 's a nonsmoker .,train,il est non-fumeur .
2,he 's smarter than me .,train,il est plus intelligent que moi .
3,he 's a lovely young man .,train,c'est un adorable jeune homme .
4,he 's three years older than me .,train,il a trois ans de plus que moi .


In [ ]:
nmt_df.to_csv(args.output_data_path)